# sentiment analysis with textblob at the movie's caption. use time interval 1 sec, find positive, negative, neutral sentiments for each video. Have a dictionary with: the title of caption, the intervals  , the sentiments

# Imports

In [147]:
import pysrt
from datetime import date, datetime, timedelta, time
import pysrt
from textblob import TextBlob
import matplotlib
from matplotlib import style
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (16.0, 9.0)
style.use('fivethirtyeight')
import os

import csv
import os
import os, re, sys
from stat import *

import numpy as np
import datetime

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import csv

from nltk.corpus import sentiwordnet as swn
nltk.download('sentiwordnet')
import re

import pickle


[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/mscuser/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


In [134]:
nltk.downloader.download('vader_lexicon')

[nltk_data] Error loading vader_lexicon: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

# Remove segments at which only tags are writtesn as caption text

In [133]:
pattern = re.compile("\[(.*?)\]|\-\[(.*?)\]")


# Handling srt Subtitles

# textblob sentiment Analysis

In [25]:
def textblob_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    return polarity
    

# Vader sentiment Analysis

In [135]:
'''VADER produces four sentiment metrics from these word ratings, 
which you can see below. The first three, positive, neutral and negative, 
represent the proportion of the text that falls into those categories. 
As you can see, our example sentence was rated as 45% positive, 
55% neutral and 0% negative. The final metric, the compound score, 
is the sum of all of the lexicon ratings (1.9 and 1.8 in this case) 
which have been standardised to range between -1 and 1. 
In this case, our example sentence has a rating of 0.69, which is pretty strongly positive.'''




sid = SentimentIntensityAnalyzer()
def vader_sentiment(text):
    polarity=sid.polarity_scores(text)['compound']
    return polarity



# SentiWordNet

In [136]:
def sentiwordnet(text):
    polarity= swn.senti_synsets(text)
    print polarity
    polarity.pos_score()
    polarity.neg_score()
    polarity.obj_score()

In [137]:

def get_sentiment(file):
    # Reading Subtitleget_sentiment
    subs = pysrt.open(file, encoding='iso-8859-1')
    n = len(subs)
    print n
    # List to store the time periods
    intervals = []
    sentiments_text_blob = []
    sentiments_vader = []
    subs_len=-1

    
    
    # Collect and combine all the text in each time interval
    for j in range(n):
        text = ""
        if (bool(pattern.match(subs[j].text_without_tags))):
            print "--------- exclude tag --------" + str(j)
            
        else:
            # Finding all subtitle text in the each time interval
            segment=subs[j].end-subs[j].start
    
            #print "end:" + str(subs[j].end)+ "strat:" +str(subs[j].start) + "segment:"+str(segment) + "segment.to_time:"+str(segment.to_time())
            intervals.append(subs[j].start + segment)
            subs_len=subs_len+1
            #for example [0,4],[4,6]->4,6
            if subs[j].end.to_time() <= (subs[j].start+ intervals[subs_len]).to_time():
                 text += subs[j].text_without_tags + " "
                
            else:
                break
            # Sentiment Analysis with TextBlob

            sentiment_blob=textblob_sentiment(text)
            sentiments_text_blob.append(sentiment_blob)

            # Sentiment Analysis with Vader
            sentiment_vader=vader_sentiment(text)
            sentiments_vader.append(sentiment_vader)
        

    #find the avrage of the 2 different sentiment analysis
    avg_sentiments=[(a_i + b_i)/float(2) for a_i, b_i in zip(sentiments_vader, sentiments_text_blob)]
    print len(avg_sentiments)
    print len(intervals)
    return (intervals, sentiments_text_blob)


In [138]:
# Utility to find average sentiment
def average(y):
    avg = float(sum(y))/len(y)
    return avg

In [139]:
#dict={id, path, , intervals, sentiments}
dict={}

i=interval_segments=sentiment_segments=0
def walktree(TopMostPath, callback):

    '''recursively descend the directory tree rooted at TopMostPath,
       calling the callback function for each regular file'''
    global dict
    for f in os.listdir(TopMostPath):
        pathname = os.path.join(TopMostPath, f)
        mode = os.stat(pathname)[ST_MODE]
        if S_ISDIR(mode):
            # It's a directory, recurse into it
            walktree(pathname, callback)
        elif S_ISREG(mode):
            # It's a file, call the callback function
            callback(pathname)
        else:
            # Unknown file type, print a message
            print 'Skipping %s' % pathname


def sentiment(file):
    
    global i 
    global interval_segments, sentiment_segments
    #the srt files we need to train our model
    if '.srt' in file:
        interval_segments, sentiment_segments = get_sentiment(file)
        #fig, ax = plt.subplots()
        #plt.plot(interval_segments,sentiment_segments)
        #plt.title(file, fontsize=32)
        #plt.ylim((-1, 1))
        #plt.ylabel("Sentiment Polarity")
        #plt.xlabel("Running Time")
        #plt.text(.5, 1.03, "Average Sentiment - " + str(round(average(y), 4)), color="green")
        
        #DICTIONARY PATH,INTERVALS,SENTIMENTS
        dict[i]={}
        dict[i]['Id']=i
        dict[i]['path']=file
        dict[i]['intervals']=interval_segments
        dict[i]['sentiments']=sentiment_segments
        i=i+1
        
        


In [140]:
path = '/home/mscuser/multimodal/multimodal_audio'
walktree(path, sentiment)


100
--------- exclude tag --------5
--------- exclude tag --------8
--------- exclude tag --------12
--------- exclude tag --------16
--------- exclude tag --------17
--------- exclude tag --------32
--------- exclude tag --------35
--------- exclude tag --------54
--------- exclude tag --------59
--------- exclude tag --------61
--------- exclude tag --------62
--------- exclude tag --------66
--------- exclude tag --------81
--------- exclude tag --------86
--------- exclude tag --------89
--------- exclude tag --------91
84
84


In [141]:
print dict[0]

{'path': '/home/mscuser/multimodal/multimodal_audio/Travis Scott Shows Off His Broadway Musical Abilities-QV9nArq7phA.en.srt', 'intervals': [SubRipTime(0, 0, 1, 634), SubRipTime(0, 0, 2, 868), SubRipTime(0, 0, 4, 604), SubRipTime(0, 0, 5, 971), SubRipTime(0, 0, 7, 707), SubRipTime(0, 0, 11, 910), SubRipTime(0, 0, 13, 746), SubRipTime(0, 0, 19, 119), SubRipTime(0, 0, 20, 419), SubRipTime(0, 0, 22, 21), SubRipTime(0, 0, 26, 92), SubRipTime(0, 0, 27, 794), SubRipTime(0, 0, 30, 563), SubRipTime(0, 0, 37, 403), SubRipTime(0, 0, 38, 604), SubRipTime(0, 0, 41, 341), SubRipTime(0, 0, 44, 144), SubRipTime(0, 0, 46, 812), SubRipTime(0, 0, 48, 914), SubRipTime(0, 0, 51, 150), SubRipTime(0, 0, 52, 618), SubRipTime(0, 0, 54, 454), SubRipTime(0, 0, 58, 357), SubRipTime(0, 1, 1, 728), SubRipTime(0, 1, 3, 729), SubRipTime(0, 1, 5, 431), SubRipTime(0, 1, 8, 801), SubRipTime(0, 1, 13, 173), SubRipTime(0, 1, 15, 141), SubRipTime(0, 1, 19, 679), SubRipTime(0, 1, 22, 515), SubRipTime(0, 1, 24, 317), SubRip

# Make a csv for all videos that contains:videoid, path, list of interval segments, list of sentiments

In [143]:
for i in range(len(dict)):
    with open('captions_polarity.csv', 'wb') as f:
        writer = csv.writer(f)
        for key, value in dict.items():
            writer.writerow([key, value])
            
            

In [144]:

#print len(sentiment_segments)
#print sentiment_segments

In [145]:
#print interval_segments[0]
#print interval_segments[1]
#print interval_segments[99]

# Make a csv for each video that contains: segment interval, pollarity for all  segments of the video.  The name of the file is polarity follwed by videoId. eg:polarity0

In [146]:
for dicts in range(len(dict)):
    new_csv='polarity'+str(dict[dicts]['Id']) +'.csv'
    with open(new_csv, 'wb') as f:
        writer = csv.writer(f)
        for j in range(len(dict[dicts]['intervals'])):
            context=(dict[dicts]['intervals'][j],dict[dicts]['sentiments'][j])
            writer.writerows([context])
            
            
            
            
            
          

# Write to pickle file

In [148]:
favorite_color = { "lion": "yellow", "kitty": "red" }

for dicts in range(len(dict)):
    context=[]
    pickle_name='polarity'+str(dict[dicts]['Id']) +'.p'
    for j in range(len(dict[dicts]['intervals'])):
        context.append((dict[dicts]['intervals'][j],dict[dicts]['sentiments'][j]))
    pickle.dump( context, open( pickle_name, "wb" ) )